In [ ]:
X = torch.randn((256,56,54)).to(devices[0])
a = torch.randn((256,2)).to(devices[0])
#net = TACModel(num_hiddens=56, norm_shape=[56], 
                 ffn_num_input=56, ffn_num_hiddens=128, num_heads=8, num_layers=12, dropout=0.01,
                 key_size=56, query_size=56, value_size=56, hid_in_features=56, t_in_features=56).to(devices[0])
a = net(X,a)[0]
a.shape

In [ ]:
total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
total_params

In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as Data
from torch import nn
import torch.optim as optim
from d2l import torch as d2l
import math
from torch.nn import functional as F
from torch.distributions import Categorical
import gym
from gym import spaces
import matplotlib.pyplot as plt

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]  

In [3]:
tr_datasets=[]
for i in range(8):
    dataset_path = f'C:/Users/Administrator/SS_data/dataset/train/traindataset{i}'
    tr_datasets += torch.load(dataset_path)
tr = tr_datasets

In [4]:
len(tr)

1024

In [5]:
def load(dataset, batch_size):
    loader = Data.DataLoader(dataset=dataset,
                         batch_size=batch_size,
                         shuffle=True)
    return loader

In [6]:
train_iter = load(tr, 256)

In [7]:
train_list = []
for i in train_iter:
    train_list.append(i)
    print(i.shape)

torch.Size([256, 2457, 54])
torch.Size([256, 2457, 54])
torch.Size([256, 2457, 54])
torch.Size([256, 2457, 54])


In [8]:
devices = d2l.try_all_gpus()
devices

[device(type='cuda', index=0)]

In [9]:
class TACModel(nn.Module):
    def __init__(self, num_hiddens, norm_shape, 
                 ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout,
                 key_size, query_size, value_size,
                 t_in_features, hid_in_features):
        super(TACModel, self).__init__()
        self.acount = AcountExtract()
        self.encoder = TACEncoder(num_hiddens, norm_shape, 
                 ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout,
                 key_size, query_size, value_size)    
        self.tap = TransiActionPred(t_in_features)
        self.tnp = TransiNumberPred(t_in_features)
        self.value = ValueNet(t_in_features)
        self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
                                    nn.Tanh())

    def forward(self, x, acount):
        Acount  = self.acount(acount)
        X = torch.cat((x,Acount), dim=2)
        encoded_X = self.encoder(X)
        action_hat = self.tap(self.hidden(encoded_X[:, :, -1]))
        number_hat = self.tnp(self.hidden(encoded_X[:, :, -1]))
        value = self.value(self.hidden(encoded_X[:, :, -1]))
        return  action_hat, number_hat, value
    
class Encoder(nn.Module):
    def __init__(self, **kwargs):
        super(Encoder, self).__init__(**kwargs)

    def forward(self, X, *args):
        raise NotImplementedError
class DotProductAttention(nn.Module):
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values):
        d = queries.shape[-1]
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        self.attention_weights = nn.functional.softmax(scores, dim=-1)
        return torch.bmm(self.dropout(self.attention_weights), values)
def transpose_qkv(X, num_heads):
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    X = X.permute(0, 2, 1, 3)
    return X.reshape(-1, X.shape[2], X.shape[3])

def transpose_output(X, num_heads):
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

class MultiHeadAttention(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values):
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        output = self.attention(queries, keys, values)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)
class PositionWiseFFN(nn.Module):
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs,
                 **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))
    
class AddNorm(nn.Module):
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.BatchNorm1d(normalized_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)
    
class EncoderBlock(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = MultiHeadAttention(key_size, query_size, value_size, num_hiddens, num_heads, dropout, use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X):
        Y = self.addnorm1(X, self.attention(X, X, X))
        return self.addnorm2(Y, self.ffn(Y))
class TACEncoder(nn.Module):
    def __init__(self,num_hiddens, norm_shape, 
                 ffn_num_input,ffn_num_hiddens, num_heads, num_layers, dropout,
                 key_size, query_size, value_size,
                 **kwargs):
        super(TACEncoder, self).__init__(**kwargs)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module(f"{i}", EncoderBlock(
                key_size, query_size, value_size, num_hiddens, norm_shape,
                ffn_num_input, ffn_num_hiddens, num_heads, dropout, True))
        self.pos_embedding = nn.Parameter(torch.randn(1, 56, num_hiddens))

    def forward(self, X):
        X = X + self.pos_embedding.data[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X)
        return X
    
class TransiActionPred(nn.Module):
    def __init__(self, num_inputs, **kwargs):
        super(TransiActionPred, self).__init__(**kwargs)
        self.output = nn.Linear(num_inputs, 3)
    def forward(self, X):
        return F.softmax(self.output(X), dim=-1)

class TransiNumberPred(nn.Module):       #---使用类别优化器，将交易数量拟合为仓位百分比[10%, 50%. 100%]
    def __init__(self, num_inputs, **kwargs):
        super(TransiNumberPred, self).__init__(**kwargs)
        self.output = nn.Linear(num_inputs, 3)
    def forward(self, X):
        return F.softmax(self.output(X), dim=-1)
    
class ValueNet(nn.Module):
    def __init__(self, num_inputs, **kwargs):
        super(ValueNet, self).__init__(**kwargs)
        self.output = nn.Linear(num_inputs, 1)
    def forward(self, X):
        return self.output(X)

class AcountExtract(nn.Module):
    def __init__(self, **kwargs):
        super(AcountExtract, self).__init__(**kwargs)
        self.expand = nn.Linear(2, 56*2)
    def forward(self, X):
        X = X.view(-1, 1, 2)
        X = self.expand(X)
        X = X.view(-1, 56, 2)
        return X
#注意：action和number的多任务分类预测损失函数需改为，两个单独任务的loss加权和，其中权重是超参数

In [10]:
class StockTradingEnv:
    def __init__(self):
        #初始化参数
        self.current_day = 0
        self.state_size = (256, 56, 54)
        self.position_options = torch.tensor([0.1, 0.5, 1.0])
        self.num_stocks = 256

        #初始化状态
        self.account_balance = torch.full((self.num_stocks,), 10000.0)
        self.stock_quantity = torch.zeros(self.num_stocks)
        
        #定义空间
        self.action_space = spaces.Tuple((spaces.Discrete(3), spaces.Discrete(3)))  # 交易行为和交易数量
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, 
                                            shape=self.state_size, dtype=np.float32)

    def load_data(self, data):
        if hasattr(self, 'data'):
            del self.data
        self.data = data
        self.total_days = data.shape[1]   #2457天
        self.reset()

    def reset(self):
        #重置环境状态
        self.current_day = 0
        self.account_balance.fill_(10000.0)
        self.stock_quantity.zero_()

    def step(self, trade_actions, position_actions):
        #获取当前状态
        current_state = self._get_current_state()
        #获取当前股价
        closing_prices = current_state[:,-1,0]
        #计算交易
        rewards = torch.zeros(self.num_stocks)
        for i in range(self.num_stocks):
            closing_price = closing_prices[i]
            trade_action = trade_actions[i]
            position_action = position_actions[i]
            if trade_action == 1:  # 买入
                buy_quantity = (self.account_balance[i] * self.position_options[position_action] / closing_price).floor()
                if buy_quantity > 0:
                    self.account_balance[i] -= buy_quantity * closing_price
                    self.stock_quantity[i] += buy_quantity
                    income = (self.account_balance[i]+self.stock_quantity[i]*closing_price - 10000)/10000
                    rewards[i] = income + 0.1
                else:
                    rewards[i] = -1       #惩罚不合规交易
            elif trade_action == 2:  # 卖出
                sell_quantity = (self.position_options[position_action] * self.stock_quantity[i]).floor()
                if sell_quantity > 0:
                    self.account_balance[i] += sell_quantity * closing_price
                    self.stock_quantity[i] -= sell_quantity
                    income = (self.account_balance[i]+self.stock_quantity[i]*closing_price - 10000)/10000
                    rewards[i] = income + 0.1
                else:
                    rewards[i] = -1       #惩罚不合规交易
            else:  # 不交易
                income = (self.account_balance[i]+self.stock_quantity[i]*closing_price - 10000)/10000
                rewards[i] = income * 0.5 - 1
        # 更新当前交易日
        self.current_day += 1
        # 检查是否达到游戏结束
        done = self.current_day >= self.total_days - self.state_size[1] - 1
        next_state = self._get_next_state()
        
        return next_state, rewards, done, _
    
    def _get_current_state(self):
        return self.data[:, self.current_day:(self.current_day + self.state_size[1]), :]
    
    def _get_next_state(self):
        start = self.current_day
        end = start + self.state_size[1]
        return self.data[:, start:end, :]
    
    def get_acount(self):
        # 返回当前状态
        return torch.stack([self.account_balance, self.stock_quantity], dim=1)

In [11]:
class DataMemory:
    def __init__(self):
        self.data = {
            'reward': [],
            'loss': []   }

    def add_data(self, reward, loss):
        self.data['reward'].append(reward)
        self.data['loss'].append(loss)

    def reset(self):
        for key in self.data.keys():
            self.data[key].clear()

    def get_data(self, key):
        return torch.stack(self.data[key])

In [12]:
policynet = TACModel(num_hiddens=56, norm_shape=[56], 
                 ffn_num_input=56, ffn_num_hiddens=128, num_heads=8, num_layers=12, dropout=0.01,
                 key_size=56, query_size=56, value_size=56, hid_in_features=56, t_in_features=56).to(devices[0])
pretrained_dict = torch.load("Transformer.params")
policynet.load_state_dict(pretrained_dict)
optimizer = optim.Adam(policynet.parameters(), lr=0.001)
gamma = 0.8
datamemory = DataMemory()
env = StockTradingEnv()
env.reset()

In [13]:
act_weight = 0.5
num_weight = 0.5
num_epoch = 1
r_list = []
l_list = []
for i in range(num_epoch):
    for i, state in enumerate(train_iter):
        #创建环境
        env.load_data(state)
        done = False
        while not done:
            #获得当前环境账户信息
            current_acount = (env.get_acount()).to(devices[0]).float()
            #获得当前状态空间
            current_state = env._get_current_state().float().to(devices[0])
            
            #进入GPU的net，获得动作空间,value
            action_prob, number_prob, value_current = policynet(current_state, current_acount)
            #根据概率分布随机抽样动作
            action_dist = Categorical(action_prob)          
            number_dist = Categorical(number_prob)          
            actions = action_dist.sample().detach()                   #(256,1)
            numbers = number_dist.sample().detach()                   #(256,1)
            #进入CPU的环境交互
            next_state, reward, done,_ = env.step(actions.cpu(), numbers.cpu())
            reward = reward.to(devices[0]).float()
            
            #获得下一环境账户信息
            next_acount =  (env.get_acount()).to(devices[0]).float()
            next_state = next_state.float().to(devices[0])
            #获得下一value
            _, _, value_next = policynet(next_state, next_acount)
            #TD算法
            td_error = reward + (gamma * value_next * (1 - int(done))) - value_current
            #log动作
            act_loss = -torch.log(action_prob[[i for i in range((len(action_prob)))],actions.long()]) * td_error
            avg_act_loss = act_loss.mean()
            num_loss = -torch.log(number_prob[[i for i in range((len(number_prob)))],numbers.long()]) * td_error
            avg_num_loss = num_loss.mean()
            #计算加权总损失
            total_loss = act_weight * avg_act_loss + num_weight * avg_num_loss
            #记录数据
            datamemory.add_data(reward.sum(), total_loss)
            
            #更新
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            
            state = next_state
            if done:
                break
        r = datamemory.get_data('reward').sum()
        l = datamemory.get_data('loss').sum()
        r_list.append(r.cpu())
        l_list.append(l.cpu())
        print(f'reward:{r:.2f}')
        print(f'loss:{l:.2f}')
        datamemory.reset()

reward:10632.30
loss:-13418.12
reward:1712.36
loss:-889.54
reward:18725.96
loss:-3909.14
reward:57542.27
loss:-3681.47


In [14]:
transformer_params = policynet.state_dict()
torch.save(transformer_params, "Transformer.params")

In [15]:
import pickle
with open('./r_list.pkl','wb') as file:
    pickle.dump(r_list, file)
with open('./l_list.pkl','wb') as file:
    pickle.dump(l_list, file)

In [ ]:
plt.plot(r_list)
plt.title("Total Rewards")
plt.show()